Wczytanie zbioru danych.

In [9]:
import pandas as pd

pd.set_option('display.max_columns', 500)

df = pd.read_csv("reviews_train.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,summary,unixReviewTime,reviewTime,score
0,A35C43YE9HU9CN,B0064X7B4A,Joan Miller,"[0, 0]",I have decided not to play this game. I can't...,Friends,1396396800,"04 2, 2014",1.0
1,AHFS8CGWWXB5B,B00H1P4V3E,WASH ST. GAMER,"[3, 4]",The Amazon Appstore free app of the day for Ju...,"Amazon Makes This ""Longest Spring Ever"" for Fi...",1402272000,"06 9, 2014",2.0
2,A3EW8OTQ90NVHM,B00CLVW82O,Kindle Customer,"[0, 4]",this game was so mush fun I wish I could play ...,best,1368921600,"05 19, 2013",5.0
3,AJ3GHFJY1IUTD,B007T9WVKM,BrawlMaster4,"[0, 2]","Its pretty fun and very good looking, but you...",Fun Game,1350172800,"10 14, 2012",5.0
4,A3JJGBS4EL603S,B00J206J5E,"K. Wilson ""thesupe""","[0, 0]",good graphics; immersive storyline; hard to st...,great game!,1396915200,"04 8, 2014",5.0


Wstępne przetwarzanie danych

Odrzucanie niepełnych wierszy, stanowiących nieznaczną częścią pełnego zbioru.

In [10]:
print("Incomplete rows: ", len(df) - len(df.dropna()))
df = df.dropna()

Incomplete rows:  46


Balansowanie zbioru danych w celu uzyskania równej reprezentacji każdej z klas.

In [11]:
from imblearn.under_sampling import RandomUnderSampler

print(df.score.value_counts() / df.shape[0] * 100, df.shape[0])

X = df[list(set(df.columns) - {"score"})]
y = df['score']
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=42)
X_res, y_res = rus.fit_resample(X, y)

X_res['score'] = y_res
df= X_res

print(df.score.value_counts() / df.shape[0] * 100, df.shape[0])
df.head()

Przetworzenie kolumny 'helpful' na format użyteczny dla klasyfikatora.

In [12]:
df['helpful'] = df['helpful'].str.replace("[", "")
df['helpful'] = df['helpful'].str.replace("]", "")
df[['helpfulP', 'helpfulN']] = df['helpful'].str.split(',', 1, expand=True)
df['helpfulP'] = pd.to_numeric(df['helpfulP'])
df['helpfulN'] = pd.to_numeric(df['helpfulN'])

C:\Users\festo\AppData\Local\Temp\ipykernel_9008\2530756455.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['helpful'] = df['helpful'].str.replace("[", "")
C:\Users\festo\AppData\Local\Temp\ipykernel_9008\2530756455.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['helpful'] = df['helpful'].str.replace("]", "")


Odrzucenie cechy reviewTime, ponieważ oznacza ona to samo co unixReviewTime a jest mniej praktyczna.

In [13]:
df.drop(["reviewTime", 'helpful'], axis=1, inplace=True)

Kodowanie cech tekstowych na liczby.

In [14]:
import json
from sklearn.preprocessing import LabelEncoder

# Kodowanie cech tekstowych na liczbowe
encoders = {}
for col in ['reviewerID', 'asin', 'reviewerName']:
    enc = LabelEncoder().fit(df[col])
    df[col] = enc.transform(df[col])
    # encoders.update({col:enc})
df.head()

,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,score,helpfulP,helpfulN
0,49575,2329,23869,I have decided not to play this game. I can't...,Friends,1396396800,1.0,0,0
1,75353,11888,50411,The Amazon Appstore free app of the day for Ju...,"Amazon Makes This ""Longest Spring Ever"" for Fi...",1402272000,2.0,3,4
2,55654,9330,27625,this game was so mush fun I wish I could play ...,best,1368921600,5.0,0,4
3,76417,4425,6657,"Its pretty fun and very good looking, but you...",Fun Game,1350172800,5.0,0,2
4,58548,12778,25658,good graphics; immersive storyline; hard to st...,great game!,1396915200,5.0,0,0


Podstawowe informacje o zbiorze

In [15]:
print("---Dataset description---\n", df.describe())
print("---Correlation Matrix---\n", df.corr())

---Dataset description---
           reviewerID           asin   reviewerName  unixReviewTime  \
count  555745.000000  555745.000000  555745.000000    5.557450e+05   
mean    43623.056312    6102.522288   35618.293870    1.368604e+09   
std     25200.616434    3631.703504   21639.569796    2.353921e+07   
min         0.000000       0.000000       0.000000    1.300752e+09   
25%     21733.000000    2882.000000   16799.000000    1.354579e+09   
50%     43564.000000    6102.000000   35062.000000    1.370650e+09   
75%     65570.000000    9056.000000   54039.000000    1.388707e+09   
max     87222.000000   13207.000000   74191.000000    1.406074e+09   

               score       helpfulP       helpfulN  
count  555745.000000  555745.000000  555745.000000  
mean        3.945410       3.704395       4.802875  
std         1.354681      39.582055      46.280171  
min         1.000000       0.000000       0.000000  
25%         3.000000       0.000000       0.000000  
50%         5.000000    

In [16]:
df.to_csv("reviews_preprocessed_ub.csv", index=False)